In [1]:
import pandas as pd

In [24]:
df = pd.read_csv("data/1000i.csv",names = ['recpt_id','quantity','item'])

In [39]:
markbaskets = {}
for row in df.values:
    item_id = row[2]
    r_id = row[0]
    
    if(r_id not in markbaskets):
        markbaskets[r_id] = []
        
    markbaskets[r_id].append(item_id)

In [67]:
mbs = [mb for mb in markbaskets.values()]
itemsets = list(set(df['item']))
apriori(mbs,itemsets,.01)

TypeError: 'NoneType' object is not iterable

In [66]:
def apriori(mbs,itemsets,minSupp):
    #Fx all items sets of length x where the support of an item set is greater
    #than the minimum support
    #all the f arrays
    fs= []
    f1 = []
    for iset in itemsets:
        if(type(iset) is not list):
            iset = [iset]
               
        supp = getSupport(iset,mbs)
        if(supp >= minSupp):
             f1.append(iset)     
    fs.append(f1)
    k = 1
    
    while(len(fs[k-1])>0):
        #The candidates for itemsets
        pcands= candGen(fs[k-1],k-1)
        #Counts the number of market baskets that contain the potential candidate
        counts_k = []
        #Initializing all counts to 0
        for c in pcands:
            counts_k.append(0)
        
        cand_idx = 0
        for mb in mbs:
            for c in pcands:
                if(isSubset(c,mb)):
                    counts_k[cand_idx]+=1
                cand_idx+=1    
            cand_idx = 0
        
        fk = []
        
        while(cand_idx < len(pcands)):
            if(counts_k[cand_idx]/len(mbs) >= minSupp):
                fk.append(pcands[cand_idx])
            cand_idx += 1
        
        fs.append(fk)   
        k += 1
                    
    
    return fs
        
        
    

In [48]:
#Returns support of itemset
#Checks how many marketbaskets contain the itemset
def getSupport(itemset,mbs):
    count = 0
    for mb in mbs:
        if isSubset(itemset,mb):
            count+=1
    return count/len(mbs)


In [51]:
#Passing in arrray of itemsets of length k
#the size/length of the item sets K
def candGen(F,k):
    candidates = []
    
    for f1 in F:
        for f2 in F:
            if( (f1 is not f2) and (len(list(set(f1).union(f2))) == len(f1)+1) ):
                candidate = list(set(f1).union(f2))
                candidates.append(candidate)
                
    for cand in candidates:
        flag = True
        for item in cand:
            prunedCand = cand.remove(item)
            if(not isSubset(prunedCand, F)):
                flag = False
        #Remove candidates that were flagged having low support
        if(not flag):
            candidates = candidates.remove(cand)
    return candidates          
                
    

In [63]:
#takes two lists, returns True iff A is subset of B
def isSubset(A, B):
    return all(a in B for a in A)